In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
import os
from keras.utils.vis_utils import plot_model

In [2]:
img_width = 224
img_height = 224
tf.random.set_seed(42)
val_size = 0.2

In [30]:
def preprocess_image(filename):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    #image = tf.image.random_jpeg_quality(image, 75, 95)
    image = tf.image.resize(image, (img_width, img_height,))
    #image = image.numpy()
    image = np.rint(image)
    return image


def preprocess_data(x):
    anchor, positive, negative = x
    return tf.stack([preprocess_image(anchor),
                     preprocess_image(positive),
                     preprocess_image(negative)], axis=0)

def create_dataset(df_filename):
    data = np.loadtxt(df_filename, dtype=str, delimiter=' ')
    #anchor_images = tf.zeros((len(data), img_width, img_height, 3), dtype=np.float32)
    #positive_images = tf.zeros((len(data), img_width, img_height, 3), dtype=np.float32)
    #negative_images = tf.zeros((len(data), img_width, img_height, 3), dtype=np.float32)
    anchor_dataset = tf.data.Dataset.from_tensor_slices([preprocess_image('food/food/'+image[0]+'.jpg') for image in data])
    positive_dataset = tf.data.Dataset.from_tensor_slices([preprocess_image('food/food/'+image[1]+'.jpg') for image in data])
    negative_dataset = tf.data.Dataset.from_tensor_slices([preprocess_image('food/food/'+image[2]+'.jpg') for image in data])
    x = tf.data.Dataset.from_tensor_slices((anchor_dataset, positive_dataset, negative_dataset))
    dataset = tf.data.Dataset.zip((x,))
    dataset = dataset.map(preprocess_data)
    return dataset


In [31]:
datas = create_dataset('train_triplets_short.txt')


TypeError: in user code:

    File "C:\Users\jonat\AppData\Local\Temp\ipykernel_1860\2605905558.py", line 15, in preprocess_data  *
        return tf.stack([preprocess_image(anchor),
    File "C:\Users\jonat\AppData\Local\Temp\ipykernel_1860\2605905558.py", line 2, in preprocess_image  *
        image_string = tf.io.read_file(filename)

    TypeError: Expected string passed to parameter 'filename' of op 'ReadFile', got <_VariantDataset element_spec=TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None)> of type '_VariantDataset' instead. Error: Expected string, but got <_VariantDataset element_spec=TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None)> of type '_VariantDataset'.


In [43]:
from keras.layers.pooling import MaxPooling2D
from keras.layers import Flatten
def makeModel():
    input = tf.keras.Input(shape=(3, img_width, img_height, 3))
    encoder = tf.keras.applications.MobileNetV3Large(include_top=False, input_shape=(img_height, img_width, 3))
    encoder.trainable = False
    decoder = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(),
                                   tf.keras.layers.Dense(128),
                                   #tf.keras.layers.Dropout(0.5),
                                   tf.keras.layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))
                                   ])
    anchor, positive, negative = tf.unstack(input, axis=0)
    anchor_features = decoder(encoder(tf.keras.applications.mobilenet_v3.preprocess_input(anchor)))
    positive_features = decoder(encoder(tf.keras.applications.mobilenet_v3.preprocess_input(positive)))
    negative_features = decoder(encoder(tf.keras.applications.mobilenet_v3.preprocess_input(negative)))
    embedding = tf.stack([anchor_features, positive_features, negative_features], axis=0)
    model = tf.keras.Model(inputs=input, outputs=embedding)
    plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
    return model


In [44]:
model = makeModel()